# Predict Winner from Realtime Stats using RNN

In [75]:
import torch
from torch import nn
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm


SEED = 233
PREDICT_TIME = 15

## Dataset

In [76]:
class LOLDataset(Dataset):
    def __init__(self,moneyData,label):
        self.money=[]
        for i in range(PREDICT_TIME):
            # self.money.append([line[i] if i < len(line) else line[-1] for line in moneyData ])
            self.money.append([line[i] for line in moneyData ])
        self.label=[i for i in label]
    
    def __getitem__(self, item):
        return torch.tensor([[torch.scalar_tensor(x[item])] for x in self.money]),torch.tensor(self.label[item])
    
    def __len__(self):
        return len(self.label)
            

In [77]:
df = pd.read_csv('../data/LeagueofLegends.csv', sep=',')
df = df[df['gamelength'] >= PREDICT_TIME]
df.reset_index(drop = True, inplace = True)
print(f'# of matches: {len(df)}')

from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
df['golddiff'].head()
# df["bResult"]

# of matches: 7620


0    [0, 0, -14, -65, -268, -431, -488, -789, -494,...
1    [0, 0, -26, -18, 147, 237, -152, 18, 88, -242,...
2    [0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ...
3    [0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ...
4    [40, 40, 44, -36, 113, 158, -121, -191, 23, 20...
Name: golddiff, dtype: object

Normalize data:

In [78]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for row in df['golddiff']:
    scaler.partial_fit(np.asanyarray(row).reshape(-1, 1))

golddiff = [scaler.transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df['golddiff']]
print(len(golddiff))

7620


## Neural Network Structure

### Recurrent Neural Network (RNN)

In [79]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        self.hidden_size = 256
        
        self.rnn= nn.RNN(
            nonlinearity = 'relu',
            input_size = 1,
            hidden_size = self.hidden_size,
            num_layers = 1,
            batch_first = True
        )

        self.out = nn.Linear(self.hidden_size, 2)
    
    def forward(self,x):
        r_out, hn = self.rnn(x, torch.zeros(1, len(x), self.hidden_size))
        out = self.out(r_out[:, -1, :])
        return out

#### Long Short-Term Memory (LSTM) Network

In [80]:
class LSTMRNN(nn.Module):
    def __init__(self):
        super(LSTMRNN, self).__init__()
        self.hidden_size = 256
        self.rnn=nn.LSTM(
            input_size = 1,
            hidden_size = self.hidden_size,
            num_layers = 1,
            batch_first = True
        )
        self.out = nn.Linear(self.hidden_size, 2)
    
    def forward(self,x):
        h0 = torch.zeros(1, len(x), self.hidden_size)
        c0 = torch.zeros(1, len(x), self.hidden_size)
        r_out,(h_n, h_c)=self.rnn(x, (h0, c0))
        out = self.out(r_out[:, -1, :])
        return out

## Training

Split the dataset into Train:Held-Out:Test = 6:2:2, so we can early stop when held-out accuracy drops.

Using a batch size of 32 to load dataset.

In [81]:
BATCH_SIZE = 32

dataset = LOLDataset(golddiff, df["bResult"])
test_size = valid_size = int(0.2 * len(dataset))
train_size = len(dataset)- test_size - valid_size

trainDataset, validDataset, testDataset = random_split(
    dataset = dataset,
    lengths = [train_size, valid_size, test_size],
    generator = torch.Generator().manual_seed(SEED)
)

trainLoader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle=True)
validLoader = DataLoader(validDataset, batch_size = BATCH_SIZE)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)

In [82]:
def train(dataloader, model, loss_fn, optimizer, mute = False):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = Variable(x), Variable(y)

        predict = model(x)
        loss = loss_fn(predict, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 30 == 0 and not mute:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [83]:
def test(dataloader, model, loss_fn, validation = False):
    model.eval()
    size = len(dataloader.dataset)

    correct = 0
    test_loss = 0
    with torch.no_grad():
        for step,(x,y) in enumerate(dataloader):
            x, y = Variable(x), Variable(y)
            predict = model(x)
            # print(predict)
            test_loss += loss_fn(predict, y).item()
            correct += (predict.argmax(1) == y).sum().item()
    
    print(f"{'Valid' if validation else 'Test'} Acc:{correct/size:>7f}, Avg Loss: {test_loss/size:>7f}")

### Training RNN

In [84]:
MUTE = False
EPOCH = 25
LR = 0.0001

torch.manual_seed(SEED)
np.random.seed(SEED)

model_RNN = RNN()
print(model_RNN)

optimizer = torch.optim.Adam(model_RNN.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model_RNN, loss_func, optimizer, mute = MUTE)
    test(validLoader, model_RNN, loss_func, validation = True)


RNN(
  (rnn): RNN(1, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
--------- Epoch #1 ---------
loss: 0.701234  [    0/ 4572]
loss: 0.670268  [  960/ 4572]
loss: 0.636943  [ 1920/ 4572]
loss: 0.609221  [ 2880/ 4572]
loss: 0.562999  [ 3840/ 4572]
Valid Acc:0.706693, Avg Loss: 0.018206
--------- Epoch #2 ---------
loss: 0.619744  [    0/ 4572]
loss: 0.592589  [  960/ 4572]
loss: 0.573755  [ 1920/ 4572]
loss: 0.535243  [ 2880/ 4572]
loss: 0.430241  [ 3840/ 4572]
Valid Acc:0.715223, Avg Loss: 0.017550
--------- Epoch #3 ---------
loss: 0.515502  [    0/ 4572]
loss: 0.549216  [  960/ 4572]
loss: 0.417349  [ 1920/ 4572]
loss: 0.575695  [ 2880/ 4572]
loss: 0.597976  [ 3840/ 4572]
Valid Acc:0.685696, Avg Loss: 0.018173
--------- Epoch #4 ---------
loss: 0.625081  [    0/ 4572]
loss: 0.629493  [  960/ 4572]
loss: 0.575935  [ 1920/ 4572]
loss: 0.669144  [ 2880/ 4572]
loss: 0.721317  [ 3840/ 4572]
Valid Acc:0.715879, Avg Loss: 0.017135
--------- Epoch #5 ---

### Training LSTM RNN

In [85]:
MUTE = False
EPOCH = 17
LR = 0.0001

torch.manual_seed(SEED)
np.random.seed(SEED)

model_LSTM = LSTMRNN()
print(model_LSTM)

optimizer = torch.optim.Adam(model_LSTM.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model_LSTM, loss_func, optimizer, mute = MUTE)
    test(validLoader, model_LSTM, loss_func, validation = True)

LSTMRNN(
  (rnn): LSTM(1, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
--------- Epoch #1 ---------
loss: 0.695953  [    0/ 4572]
loss: 0.694263  [  960/ 4572]
loss: 0.681436  [ 1920/ 4572]
loss: 0.656316  [ 2880/ 4572]
loss: 0.644940  [ 3840/ 4572]
Valid Acc:0.667323, Avg Loss: 0.019068
--------- Epoch #2 ---------
loss: 0.677252  [    0/ 4572]
loss: 0.598819  [  960/ 4572]
loss: 0.669931  [ 1920/ 4572]
loss: 0.649231  [ 2880/ 4572]
loss: 0.534009  [ 3840/ 4572]
Valid Acc:0.700131, Avg Loss: 0.018008
--------- Epoch #3 ---------
loss: 0.534779  [    0/ 4572]
loss: 0.567087  [  960/ 4572]
loss: 0.490719  [ 1920/ 4572]
loss: 0.552065  [ 2880/ 4572]
loss: 0.625606  [ 3840/ 4572]
Valid Acc:0.705381, Avg Loss: 0.017745
--------- Epoch #4 ---------
loss: 0.492611  [    0/ 4572]
loss: 0.526867  [  960/ 4572]
loss: 0.533052  [ 1920/ 4572]
loss: 0.487172  [ 2880/ 4572]
loss: 0.711388  [ 3840/ 4572]
Valid Acc:0.704068, Avg Loss: 0.017714
--------- Epoch #

## Test

Result for both models:

In [86]:
print("RNN:")
test(testLoader, model_RNN, loss_func)

print("LSTM RNN:")
test(testLoader, model_LSTM, loss_func)

RNN:
Test Acc:0.725066, Avg Loss: 0.016994
LSTM RNN:
Test Acc:0.729003, Avg Loss: 0.017034
